## Trabalho Prático #2: Planejamento e navegação ##
#### Integrantes: ####
* Filipe Rodrigues Batista de Oliveira 
* Lucas Avelar 

In [36]:
import time
import numpy as np
from matplotlib import pyplot as plt

import sim

### Parte I - 

### Parte II - Campos Potenciais.

Primeiramente vamos definir algumas constantes

In [51]:
DIST_LIMITE_OBJ = 0.05 
RAIO_INFLUENCIA_OBS = 5

Agora, implementamos as funções que vão calcular as forças de repulsão e de atração, sendo combinados posteriormente no código principal.

In [ ]:
def calcular_forca_rep(q, obstaculos, k_rep = 0.1):
    if obstaculos.size == 0:
        return 0
    else:
        diff_pos = q - obstaculos[:, 0:2]
        d = np.linalg.norm(diff_pos, axis=1)

        forca_rep = (1/d**2)*((1/d)-(1/RAIO_INFLUENCIA_OBS))*(diff_pos/d)    

        invalido = np.squeeze(d > RAIO_INFLUENCIA_OBS)
        forca_rep[invalido, :] = 0

        return k_rep * forca_rep 

In [56]:
def calcular_forca_attr(q, obj, k_attr=0.2):
    forca_attr = k_attr * (obj - q)
    
    return forca_attr

Finalmente o código principal

In [61]:
sim.simxFinish(-1)
id_cliente = sim.simxStart("127.0.0.1", 19999, True, True, 50000, 5)

if id_cliente != -1:
    # Dimensões do robô.
    L = 0.331
    r = 0.09751
    
    robo_nome = "Pioneer_p3dx"
    rv, robo = sim.simxGetObjectHandle(id_cliente, robo_nome, sim.simx_opmode_oneshot_wait)
    rv, roda_dir = sim.simxGetObjectHandle(id_cliente, robo_nome + "_rightMotor", sim.simx_opmode_oneshot_wait)
    rv, roda_esq = sim.simxGetObjectHandle(id_cliente, robo_nome + "_leftMotor", sim.simx_opmode_oneshot_wait)
    rv, sonar_frente = sim.simxGetObjectHandle(id_cliente, robo_nome + "_ultrasonicSensor5", sim.simx_opmode_oneshot_wait)
    rv, sonar_direita = sim.simxGetObjectHandle(id_cliente, robo_nome + "_ultrasonicSensor7", sim.simx_opmode_oneshot_wait)
    rv, sonar_esquerda = sim.simxGetObjectHandle(id_cliente, robo_nome + "_ultrasonicSensor1", sim.simx_opmode_oneshot_wait)
    
    rv, obj = sim.simxGetObjectHandle(id_cliente, "Goal", sim.simx_opmode_oneshot_wait)
    rv, pos_obj = sim.simxGetObjectPosition(id_cliente, obj, -1, sim.simx_opmode_oneshot_wait)
    pos_obj = pos_obj[:2]
    
    
    # Lembrar de habilitar o Real-time Mode.
    t_inicio = time.time()
    t_passou = t_inicio
    
    t = 0
    while True:
        t_atual = time.time()
        dt = t_atual - t_passou
        
        rv, pos = sim.simxGetObjectPosition(id_cliente, robo, -1, sim.simx_opmode_oneshot_wait)
        pos = np.array(pos[:2])
        rv, orientacao = sim.simxGetObjectOrientation(id_cliente, robo, -1, sim.simx_opmode_oneshot_wait)
        orientacao = orientacao[2]
        
        # Chegou no objetivo?
        if np.linalg.norm(pos_obj - pos) <= DIST_LIMITE_OBJ:
            break
        
        rv, estado_sonar_frente, pontos_frente, *_ = sim.simxReadProximitySensor(id_cliente, sonar_frente, sim.simx_opmode_oneshot_wait)
        rv, estado_sonar_dir, pontos_dir, *_ = sim.simxReadProximitySensor(id_cliente, sonar_direita, sim.simx_opmode_oneshot_wait)
        rv, estado_sonar_esq, pontos_esq, *_ = sim.simxReadProximitySensor(id_cliente, sonar_esquerda, sim.simx_opmode_oneshot_wait)
        
        tem_obstaculo_frente = (estado_sonar_frente and np.linalg.norm(pontos_frente) < RAIO_INFLUENCIA_OBS)
        tem_obstaculo_dir = (estado_sonar_dir and np.linalg.norm(pontos_dir) < RAIO_INFLUENCIA_OBS)
        tem_obstaculo_esq = (estado_sonar_esq and np.linalg.norm(pontos_esq) < RAIO_INFLUENCIA_OBS)
        
        # Posso ter um obstáculo em todas as direções. 
        obstaculos = np.empty((0,2))
        if tem_obstaculo_frente:
            np.vstack((obstaculos, np.array([pontos_frente[:2]])))
        if tem_obstaculo_dir:
            np.vstack((obstaculos, np.array([pontos_dir[:2]])))
        if tem_obstaculo_esq:
            np.vstack((obstaculos, np.array([pontos_esq[:2]])))
            
        
        # Campos potenciais 
        dx, dy = calcular_forca_attr(pos, pos_obj) + calcular_forca_rep(pos, obstaculos)
        
        # Controlador
        kr, kt = 1, 2
        
        v = kr*(dx*np.cos(orientacao) + dy*np.sin(orientacao))
        w = kt*(np.arctan2(dy,dx) - orientacao)
        
        
        # Modelo cinemática inversa.
        v_roda_dir = ((2 * v) + (w * L)) / (2 * r)
        v_roda_esq = ((2 * v) - (w * L)) / (2 * r)

        sim.simxSetJointTargetVelocity(id_cliente, roda_dir, v_roda_dir, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(id_cliente, roda_esq, v_roda_esq, sim.simx_opmode_oneshot_wait)
        
        t += dt
        t_passou = t
    
    # Parar o robô no objetivo
    sim.simxSetJointTargetVelocity(id_cliente, roda_dir, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(id_cliente, roda_esq, 0, sim.simx_opmode_oneshot_wait)
    
    sim.simxFinish(id_cliente) 

else:
    print("Falha ao conectar-se ao CoppeliaSim.")

False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 